<a href="https://colab.research.google.com/github/bubinsabu/datasets_classification/blob/main/CIFAR_10_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load and Normalize

In [10]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.transforms import transforms
from torchsummary import summary
import sys

# Define Data transformation for Normalisation

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5,0.5,0.5], std = [0.5,0.5,0.5])
])

In [4]:
# load and training CIFAR-10 datasets
trainset = torchvision.datasets.CIFAR10(root = "./data", train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = False)

# load and testing CIFAR-10 datasets
testset = torchvision.datasets.CIFAR10(root = "./data", train = False , download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = False)

classes = ("airplane", "automobile", "bird", "car","deer", "dog", "frog", "horse", "ship", "truck")

100%|██████████| 170498071/170498071 [00:06<00:00, 26654053.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
print("Train_Data_Shape: ",trainset.data.shape)
print("Test_Data_Shape: ",testset.data.shape)

Train_Data_Shape:  (50000, 32, 32, 3)
Test_Data_Shape:  (10000, 32, 32, 3)


# Define CNN

In [13]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 16,kernel_size = 3, stride = 1, padding = 1)
    self.relu1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1)
    self.relu2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(32 * 8 * 8, 128)
    self.relu3 = nn.ReLU()
    self.fc2 = nn.Linear(128, 10)



  def forward(self, x):
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.relu2(x)
    x = self.pool2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.relu3(x)
    x = self.fc2(x)
    return x

model = Net()

In [14]:
input_shape = (3, 32,32)
summary(model, input_size = input_shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
         MaxPool2d-3           [-1, 16, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]           4,640
              ReLU-5           [-1, 32, 16, 16]               0
         MaxPool2d-6             [-1, 32, 8, 8]               0
           Flatten-7                 [-1, 2048]               0
            Linear-8                  [-1, 128]         262,272
              ReLU-9                  [-1, 128]               0
           Linear-10                   [-1, 10]           1,290
Total params: 268,650
Trainable params: 268,650
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.44
Params size (MB): 1.02
Estimated T

# Define Loss Function and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

# Train the Model

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [17]:
for epoch in range(10):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 2000 == 1999:
      print(f"[Epoch {epoch + 1},Iteration {i + 1}]loss:{running_loss / 2000:.3f}")
      running_loss = 0.0
      sys.stdout.flush()

In [18]:
print(f"[Epoch {epoch + 1},Iteration {i + 1}]loss:{running_loss / 2000:.3f}")

[Epoch 10,Iteration 782]loss:0.901
